In [2]:
import pandas as pd 
import numpy as np

In [1]:
from pycaret.datasets import get_data
data = get_data('electrical_grid')

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [3]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, num_inputs=12, num_units_d1=200, num_units_d2=100):
        super(Net, self).__init__()
        self.dense0 = nn.Linear(num_inputs, num_units_d1)
        self.nonlin = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units_d1, num_units_d2)
        self.output = nn.Linear(num_units_d2, 2)
        self.softmax = nn.Softmax(dim=-1)
    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense1(X))
        X = self.softmax(self.output(X))
        return X

In [6]:
from skorch import NeuralNetClassifier
net = NeuralNetClassifier(
    module=Net,
    max_epochs=30,
    lr=0.1,
    batch_size=32,
    train_split=None
)

In [7]:
from skorch.helper import DataFrameTransformer
from sklearn.pipeline import Pipeline
nn_pipe = Pipeline(
    [
        ("transform", DataFrameTransformer()),
        ("net", net),
    ]
)

In [8]:
from pycaret.classification import *
target = "stabf"
clf1 = setup(data = data, 
            target = target,
            train_size = 0.8,
            fold = 5,
            session_id = 123,
            log_experiment = True, 
            experiment_name = 'electrical_grid_1', 
            silent = True)

,Description,Value
0,session_id,123
1,Target,stabf
2,Target Type,Binary
3,Label Encoded,"stable: 0, unstable: 1"
4,Original Data,"(10000, 13)"
5,Missing Values,False
6,Numeric Features,12
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [9]:
model = create_model("rf")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9244,0.9796,0.9667,0.9189,0.9422,0.8331,0.8353
1,0.9275,0.9793,0.9549,0.9330,0.9438,0.8417,0.8422
2,0.9225,0.9810,0.9608,0.9211,0.9406,0.8294,0.8309
3,0.9081,0.9738,0.9461,0.9130,0.9293,0.7983,0.7993
4,0.9044,0.9738,0.9471,0.9071,0.9267,0.7894,0.7909
Mean,0.9174,0.9775,0.9551,0.9186,0.9365,0.8184,0.8197
SD,0.0093,0.0031,0.0079,0.0087,0.0071,0.0206,0.0206


In [11]:
skorch_model = create_model(nn_pipe)

IntProgress(value=0, description='Processing: ', max=4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,19:15:48
Status,. . . . . . . . . . . . . . . . . .,Finalizing Model
Estimator,. . . . . . . . . . . . . . . . . .,NeuralNetClassifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


RuntimeError: expected scalar type Long but found Int